In [ ]:
#!/usr/bin/env python
# Copyright (c) Meta Platforms, Inc. and affiliates.
#
# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.

# Implementation is based on original Bounce Flash Lidar repo:
# https://github.com/co24401/BounceFlashLidar

In [ ]:
import matplotlib.pyplot as plt 
import math
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objs as go
import time
from copy import deepcopy
from math import sqrt, cos, sin, tan, atan, radians

from ray_tracing import visualize_probablistic 

In [ ]:
class voxelGrid():
    def __init__(self, x_lims, y_lims, z_lims, num_x, num_y, num_z):
        self.x_lims = x_lims
        self.y_lims = y_lims
        self.z_lims = z_lims
        self.num_x = num_x
        self.num_y = num_y
        self.num_z = num_z
        self.volume = np.zeros((num_x, num_y, num_z))

In [ ]:
fname = "chair"
data = np.load("../logs/bfshadows_" + fname + "_100.npy", allow_pickle=True).item()
outside_voxel = data["outside_voxel"]
inside_voxel = data["inside_voxel"]
PO = data["PO"]
testDets = data["testDets"]
testLas = data["testLas"]
params = data["params"]
num_x = params[0]
num_y = params[1]
num_z = params[2]
eta = params[3]
xi = params[4]
p0 = params[5]
p1 = params[6]

fin_pc = np.load("../logs/bfpc_{}.npy".format(fname), allow_pickle=True).item()['pc']
fin_pc_flattened = np.load("../logs/bfpc_{}.npy".format(fname), allow_pickle=True).item()['flattened']

In [ ]:
T = 0.5 # Probability threshold to decide that voxel is occupied --> usually only play with T

volume, probability_volume = visualize_probablistic(inside_voxel, outside_voxel, PO, T, testLas, testDets, False)

voxel_x_size = (inside_voxel.x_lims[1] - inside_voxel.x_lims[0]) / inside_voxel.num_x
voxel_y_size = (inside_voxel.y_lims[1] - inside_voxel.y_lims[0]) / inside_voxel.num_y
voxel_z_size = (inside_voxel.z_lims[1] - inside_voxel.z_lims[0]) / inside_voxel.num_z
fill_indices = np.nonzero(volume)

xx = inside_voxel.x_lims[0] + voxel_x_size * (fill_indices[0])
yy = inside_voxel.y_lims[0] + voxel_y_size * (fill_indices[1])
zz = inside_voxel.z_lims[0] + voxel_z_size * (fill_indices[2])

In [ ]:
# VISUALIZE POINT CLOUD --> ADJUST T UNTIL IT LOOKS GOOD
testShape = np.transpose(np.stack((xx, yy, zz), axis=0))  
plotly.offline.init_notebook_mode()
# Configure the trace.
points_from_2b = go.Scatter3d(
    x=np.ndarray.flatten(fin_pc_flattened[:,0]),  # <-- Put your data instead
    y=np.ndarray.flatten(fin_pc_flattened[:,1]),  # <-- Put your data instead
    z=np.ndarray.flatten(fin_pc_flattened[:,2]),  # <-- Put your data instead
    mode='markers',
    marker={
        'size': 2,
        'opacity': 1,
    }
)

points_from_shadow = go.Scatter3d(
    x=np.ndarray.flatten(xx), 
    y=np.ndarray.flatten(yy),  
    z=np.ndarray.flatten(zz),  
    mode='markers',
    marker={
        'size': 2,
        'opacity': 1,
    }
)

# Configure the layout.
layout = go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0}
)
data = [points_from_shadow]
plot_figure = go.Figure(data=data, layout=layout)

# Render the plot.
camera = dict(
    up=dict(x=0, y=1, z=0),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=-0.7, y=1.2, z=-2)
)
plot_figure.update_layout(scene_camera=camera)
plotly.offline.iplot(plot_figure)

In [ ]:
# SAVE FINAL POINT CLOUD
shadow_pc = np.stack([xx,yy,zz],axis=1)
all_points = np.concatenate([fin_pc_flattened, shadow_pc],axis=0)
np.save("../logs/bf_final_pc_{}.npy".format(fname), all_points)
print("Saved {} with points {}.".format(fname, all_points.shape))